In [23]:
from pytorch_lightning import LightningModule
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [27]:
import yaml
from transformers import T5Config
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Import Configuration
def load_model_config(config_path="configs/t5_small_config.yaml"):
    with open(config_path, "r") as file:
        config_dict = yaml.safe_load(file)
    return T5Config(**config_dict)


config = load_model_config()

FileNotFoundError: [Errno 2] No such file or directory: 'configs/t5_small_config.yaml'

In [24]:
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")

In [25]:
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

In [18]:
loss = model(input_ids=input_ids, labels=labels).loss

In [20]:
loss.item()

5.1461687088012695